In [1]:
import word_tokenizer
import pandas as pd
from gensim import models

### Load climate-related FT meetings

In [3]:
df = pd.read_pickle("../../climate_classifier/scored_data/ft_meetings_all_2.3.pkl")
df_c = df[df["y_pred"]=="climate"].reset_index(drop=True)
print(len(df_c))
print(len(df))
df_c.head(1)

17226
335170


,speaker,politician,title,party,text,date,y_pred
0,Statsministeren Helle Thorning-Schmidt,Helle Thorning-Schmidt,Statsministeren,(S),Og vi får flere arbejdspladser. Vi har beslutt...,2012-10-02,climate


In [4]:
## Add a comma + space in end of all texts
df_c["text"] = df_c["text"]+"<> "
df_c["politician"] = df_c["politician"].str.strip()
pol_c = df_c.groupby(['politician'])['text'].sum()
pol_c

politician
Aaja Chemnitz Larsen     I det sidste års tid har der været rigtig mege...
Aki-Matilda Høegh-Dam    Tak. Først tak for den her redegørelse, som er...
Aleqa Hammond            Dette lykkedes fint i den forrige regerings ti...
Alex Ahrendtsen          Det vil være en lillebitte afgift for boligeje...
Alex Vanopslagh          Tak til ordføreren for en passioneret tale om ...
                                               ...                        
Victoria Velasquez       Tak for spørgsmålet. Jeg synes egentlig ikke, ...
Villum Christensen       Tak for det. Liberal Alliance synes, at intent...
Villy Søvndal            Det er, når vi snart er mere tydeligt til sted...
Zenia Stampe             Hvis vi ikke var med i EU, ville vi jo alligev...
Øjvind Vilsholm          .Hvilke tiltag vil regeringen så tage, hvis EU...
Name: text, Length: 325, dtype: object

In [5]:
df_pol_c = pd.DataFrame(pol_c)
df_pol_c["politician"] = df_pol_c.index
df_pol_c = df_pol_c[["politician", "text"]].reset_index(drop=True)
print(len(df_pol_c.loc[4]["text"]))
df_pol_c

39835


,politician,text
0,Aaja Chemnitz Larsen,I det sidste års tid har der været rigtig mege...
1,Aki-Matilda Høegh-Dam,"Tak. Først tak for den her redegørelse, som er..."
2,Aleqa Hammond,Dette lykkedes fint i den forrige regerings ti...
3,Alex Ahrendtsen,Det vil være en lillebitte afgift for boligeje...
4,Alex Vanopslagh,Tak til ordføreren for en passioneret tale om ...
...,...,...
320,Victoria Velasquez,"Tak for spørgsmålet. Jeg synes egentlig ikke, ..."
321,Villum Christensen,"Tak for det. Liberal Alliance synes, at intent..."
322,Villy Søvndal,"Det er, når vi snart er mere tydeligt til sted..."
323,Zenia Stampe,"Hvis vi ikke var med i EU, ville vi jo alligev..."


### Vectorize texts

In [8]:
MODEL_FILE = '../../../../Models/word2vec/dsl_skipgram_2020_m5_f500_epoch2_w5.models/dsl_skipgram_2020_m5_f500_epoch2_w5.model'
model = models.Word2Vec.load(MODEL_FILE)

        ## Takes 2 min to load on Jonathan PC 2

## Used for vectorizing the description text (filtered)
words = set(model.wv.index_to_key)

In [9]:
## Remove stopwords and create lists containing words
df_pol_c['text_clean'] = df_pol_c['text'].apply(lambda x: word_tokenizer.tokenize(x))
df_pol_c.head(3)

,politician,text,text_clean
0,Aaja Chemnitz Larsen,I det sidste års tid har der været rigtig mege...,"[sidste, år, tid, rigtig, fokus, arktis, grønl..."
1,Aki-Matilda Høegh-Dam,"Tak. Først tak for den her redegørelse, som er...","[tak, først, tak, redegørelse, basere, status,..."
2,Aleqa Hammond,Dette lykkedes fint i den forrige regerings ti...,"[lykkes, finte, forrige, regering, tid, håbe, ..."


In [10]:
import numpy as np
pol_vect = np.array([np.array([model.wv[i] for i in ls if i in words])
                         for ls in df_pol_c["text_clean"]])
                         ## Takes 32 sec on Jonathan PC 2

C:\Users\COBOD\AppData\Local\Temp/ipykernel_13284/2123071034.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pol_vect = np.array([np.array([model.wv[i] for i in ls if i in words])


In [11]:
pol_vec_avg = []
for v in pol_vect:
    if v.size:
        pol_vec_avg.append(v.mean(axis=0))
    else:
        pol_vec_avg.append(np.zeros(100, dtype=float))

        ## Takes 0.4 sec Jonathan PC 2

In [12]:
df_pol_c["vec"] = pol_vec_avg
df_pol_c.head(1)

,politician,text,text_clean,vec
0,Aaja Chemnitz Larsen,I det sidste års tid har der været rigtig mege...,"[sidste, år, tid, rigtig, fokus, arktis, grønl...","[-0.10230713, 0.023683876, 0.079156294, -0.159..."


In [13]:
### Test politicians 
## Upper bound: Lars Christian Lilleholt, Mai Villadsen
## Not so much: Mette Gjerskov, Karen Hækkerup

up1 = df_pol_c[df_pol_c["politician"]=="Lars Christian Lilleholt"].reset_index(drop=True)
up2 = df_pol_c[df_pol_c["politician"]=="Mai Villadsen"].reset_index(drop=True)
bot1 = df_pol_c[df_pol_c["politician"]=="Mette Gjerskov"].reset_index(drop=True)
bot2 = df_pol_c[df_pol_c["politician"]=="Karen Hækkerup"].reset_index(drop=True)

print(model.wv.most_similar(positive=[up1["vec"][0]])[:5])
print(model.wv.most_similar(positive=[up2["vec"][0]])[:5])
print(model.wv.most_similar(positive=[bot1["vec"][0]])[:5])
print(model.wv.most_similar(positive=[bot2["vec"][0]])[:5])


[('retsundtagelse', 0.8501923084259033), ('2025aftale', 0.849542498588562), ('libyenstrategi', 0.8495377898216248), ('fossiltfrit', 0.8486163020133972), ('lidelsesreduktion', 0.8483492136001587)]
[('alenejeg', 0.8445197343826294), ('licensmuligheder', 0.8414653539657593), ('supertrist', 0.8411604762077332), ('allerallerhelst', 0.8409302234649658), ('dobbeltstraffe', 0.840675950050354)]
[('upatruljeret', 0.8430442810058594), ('alenejeg', 0.8399696350097656), ('muntjaken', 0.8393654227256775), ('0skabeloner', 0.8391795754432678), ('dobbeltstraffe', 0.8378236889839172)]


KeyError: 0

In [ ]:
print(model.wv.cosine_similarities(up1["vec"][0], [up2["vec"][0]])) ## Mai
print(model.wv.cosine_similarities(up1["vec"][0], [bot1["vec"][0]])) ## Mette G
print(model.wv.cosine_similarities(up1["vec"][0], [bot2["vec"][0]]) )## Karen Hæk

[0.97670025]
[0.968652]
[0.9421233]


In [39]:
df_votes_c = pd.read_pickle("./scikitv2/c_all_trans.pkl")
df_votes_c.head(1)

,party,politician,vote,description,date,vote_caller,y_pred,set,vote_id,Socialdemokratiet_weight_for,Socialdemokratiet_weight_against,Venstre_weight_for,Venstre_weight_against,vec,vec2,w_for,w_against,party2,vote_caller2
1,Dansk Folkeparti (DF),Alex Ahrendtsen,For,"Folketinget konstaterer, at den hidtil førte e...",2015-12-17,['Mikkel Dencker (DF)'],climate,train,"Afstemning nr. 183, 2015-16",0.570332,0.429668,0.079625,0.920375,[-1.04358420e-01 -1.38448412e-02 7.39112943e-...,"[-0.10435842, -0.0138448412, 0.0739112943, -0....",0.079625,0.920375,(DF),(DF)


In [45]:
vote1 = df_votes_c.loc[1]["vec2"]
print(model.wv.cosine_similarities(vote1, [up1["vec"][0]]))
print(model.wv.cosine_similarities(vote1, [up2["vec"][0]]))
print(model.wv.cosine_similarities(vote1, [bot1["vec"][0]]))
print(model.wv.cosine_similarities(vote1, [bot2["vec"][0]]))

[0.93099058]
[0.87828027]
[0.86871]
[0.84866758]


In [14]:
df_pol_c
df_pol_c.to_pickle("pol2vec.2.3.pkl")